In [ ]:
# default_exp aoc
from nbdev import *
from nbdev.showdoc import *

# Advent of Code Utils

> A collection of somewhat handy functions to make your AoC puzzle life solving a bit easier

In [ ]:
#export
import pandas as pd
from collections import namedtuple

Dim = namedtuple('Dim',['min','max','range'])
def dimensions(obj): 
    """
     takes an iterable of iterables and returns a namedtuple with minima, maxima and range
     for example a 2d numpy array
     dim.min, dim.max and dim.range
    """
    minim = tuple(min(obj,key = lambda x:x[i])[i] for i in range(len(obj[0])))
    maxim = tuple(max(obj,key = lambda x:x[i])[i] for i in range(len(obj[0])))# max for dimensions
    ranges = tuple(maxim[i] - minim[i]  for i in range(len(obj[0])))
    res = Dim(minim,maxim,ranges)
    return res

In [ ]:
assert dimensions([[1,2,3],[10,9,8]]) == Dim(min=(1, 2, 3), max=(10, 9, 8), range=(9, 7, 5))

Example

In [ ]:
out = dimensions([[1,2,3],[10,9,8]])
out.min

(1, 2, 3)

In [ ]:
#export
def positive(*args): 
    """ 
        takes 1 or multiple lists of n coordinates and returns it normalized (getting rid of negatives)
    """
    dtype = type(args[0][0]) # support list(s) of lists and list(s) of tuples
    if len(args)==1: # only 1 argument passed
        dim = dimensions(args[0])
        obj = args[0]
        if dtype == tuple:
            return [tuple(o[i]-dim.min[i] for i in range(len(obj[0]))) for o in obj]
        if dtype == list:
            return [[o[i]-dim.min[i] for i in range(len(obj[0]))] for o in obj]
        else: print('no support for dtype',dtype)
    else: # multiple arguments passed
        dim = dimensions([i for a in args for i in a])
        if dtype == tuple:
            return ([tuple(o[i]-dim.min[i] for i in range(len(obj[0]))) for o in obj] for obj in args)

        if dtype == list: 
            return ([[o[i]-dim.min[i] for i in range(len(obj[0]))] for o in obj] for obj in args)
        else: print('no support for dtype',dtype)


positive() will only make changes along axis where negative values are detected

In [ ]:
assert positive([(0,0,0,-4),(0,0,-10,0),(0,0,0,0)]) == [(0, 0, 10, 0), (0, 0, 0, 4), (0, 0, 10, 4)]

In [ ]:
#export
def manhattan(x,y):
    return abs(x[0]-y[0])+abs(x[1]-y[1])

In [ ]:
assert manhattan((10,10),(-1,11)) == 12

In [ ]:
#export

def binarysearch(minim,maxim,function, flips_to_true=True): 
    """
     function needs to return a boolean whether the solution is ok
     this implementation is for function that starts with false for minim and flip to true
     for TTTTFFFF, pass set flips_to_true flag to false. This flag is important to set correct!
    """
    new = minim
    while True:
        new = (minim+maxim)//2
        print(f'to_test: {new}, min {minim}, max {maxim} ', end=' ')
        res = function(new)
        print('function returns', res)
        if not flips_to_true: res = not res
        if res:
            if new == maxim: # solution found
                if flips_to_true:
                    print('solution found',new)
                    return new
                else:
                    print('solution found',new-1)
                    return new-1
            maxim = new
        else: minim = new+1


In [ ]:
assert binarysearch(0,200, lambda x: x > 50) == 51

to_test: 100, min 0, max 200  function returns True
to_test: 50, min 0, max 100  function returns False
to_test: 75, min 51, max 100  function returns True
to_test: 63, min 51, max 75  function returns True
to_test: 57, min 51, max 63  function returns True
to_test: 54, min 51, max 57  function returns True
to_test: 52, min 51, max 54  function returns True
to_test: 51, min 51, max 52  function returns True
to_test: 51, min 51, max 51  function returns True
solution found 51


In [ ]:
assert binarysearch(0,200, lambda x: x < 50, flips_to_true=False) == 49

to_test: 100, min 0, max 200  function returns False
to_test: 50, min 0, max 100  function returns False
to_test: 25, min 0, max 50  function returns True
to_test: 38, min 26, max 50  function returns True
to_test: 44, min 39, max 50  function returns True
to_test: 47, min 45, max 50  function returns True
to_test: 49, min 48, max 50  function returns True
to_test: 50, min 50, max 50  function returns False
solution found 49


In [ ]:
#export
from collections import deque
def bfs(connections, start, goal=None):
    """
    Requires a connections dict with tuples with neighbors per node.
    Or a connections function returning neighbors per node

    Returns
    if goal == None:    return dict of locations with neighbor closest to start
    elif goal found:    returns path to goal
    else:               returns False
    """
    seen = set() # the locations that have been explored
    frontier = deque([start]) # the locations that still need to be visited
    # paths = {start: [start]}
    isfunction = callable(connections)
    parents = {start: None}

    def get_path(parents,start,goal):
        # print(start,goals)
        cur = goal
        path = [cur]
        while cur != start:
            cur = parents[cur]
            path.append(cur)
        path.reverse()
        return path

    while frontier:
        search = frontier.popleft()
        if isfunction: neighbors = connections(search)
        else: neighbors = connections.get(search,None)
        if neighbors:
            for n in neighbors:
                if n not in seen:
                    seen.add(n)
                    frontier.append(n)
                    # paths[n] = paths[search]+[n]
                    parents[n]= search
                    if goal and n == goal:
                        # print('goal found')
                        return get_path(parents,start,goal)
                        # return paths[goal],parents
        seen.add(search)
    if goal: return False
    else: return parents

In [ ]:
def test_bfs(input):
    if input < 0: return (0,)
    elif input > 25: return (25,)
    else:
        return (input-1, input+1, input + 20, input -20)
bfs(test_bfs, 0,goal=10) == [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

False

In [ ]:
#hide
from nbdev.export import notebook2script; 

Converted 00_core.ipynb.
Converted index.ipynb.
converting: d:\Documenten\GitHub\adventofcode\aoc-utils\00_core.ipynb
converting d:\Documenten\GitHub\adventofcode\aoc-utils\index.ipynb to README.md


In [ ]:
#hide
notebook2script()
!nbdev_build_docs
!git commit -am "changed lib into aocutils"
!git push

Converted 00_core.ipynb.
Converted index.ipynb.
converting: d:\Documenten\GitHub\adventofcode\aoc-utils\00_core.ipynb
An error occurred while executing the following cell:
------------------
from nbdev.showdoc import show_doc
from aoc_utils.aoc import *
------------------

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<ipython-input-1-573b5a9701ce> in <module>
      1 from nbdev.showdoc import show_doc
----> 2 from aoc_utils.aoc import *

ModuleNotFoundError: No module named 'aoc_utils'
ModuleNotFoundError: No module named 'aoc_utils'

Conversion failed on the following:
00_core.ipynb
converting d:\Documenten\GitHub\adventofcode\aoc-utils\index.ipynb to README.md
[master 1022b5c] changed lib into aoc_utils
 5 files changed, 24 insertions(+), 9 deletions(-)
To https://github.com/jvanelteren/aoc-utils.git
   8e7b744..1022b5c  master -> master
